Business Context:

As a business leader at Orion Tech Solutions, you (“Alex Carter”) oversee multiple software development and IT infrastructure projects. Your responsibilities include coordinating with stakeholders, managing escalations, and ensuring timely deliveries. With hundreds of emails flooding your inbox daily, manually sorting through them is time-consuming and increases the risk of missing critical updates, client escalations, or project approvals.

Objective:
The goal of this project is to develop a Generative AI-powered system that:
✅ Summarizes emails into actionable insights using the Yesterbox approach (excluding today’s emails).
✅ Prioritizes emails based on urgency, sender, and context.
✅ Draft context-aware responses to reduce manual effort.
✅ Evaluate the drafted context-aware responses using LLM-as-a-Judge.
Tasks & Workflow
Task 1: Generate a Detailed Summary of Yesterday’s Inbox
Task 1A: Executive Dashboard (Top-Level Summary of Yesterday’s Emails)
Sample Output:
🔹 Total Emails from Yesterday: 100
🔹 🛑 Urgent & High-Priority Emails: 10 (Require Immediate Action Today)
🔹 ⚡ Deadline-Driven Emails: 8 (Must Be Addressed Today)
🔹 📌 Routine Updates & Check-ins: 35 (Review & Acknowledge)
🔹 📎 Non-Urgent & Informational Emails: 45 (Can Be Deferred or Delegated)
🔹 🎉 Personal & Social Emails: 22 (Optional Review)
🔹 🗑️ Spam/Unimportant Emails Filtered Out: 20

AI Conclusion:
"You have 18 critical emails from yesterday that require action today. Additionally, there are 35 updates to review at your convenience."

Task 1B: Analyze Urgent & High-Priority Emails (🛑 Must-Do First Today)
Focus on emails that require immediate action and impact critical projects or client relationships.

Task 1C: Review Deadline-Driven Emails (⚡ Needs Attention Today)
Identify emails tied to important deadlines and ensure timely responses.

Task 2: AI-Generated Response Drafts for Critical Email
For each Urgent & High-priority email or Deadline-Driven email from yesterday, generate an AI-powered response draft for quick review and editing before sending.

NOTE : Critical Emailsare the combination of Urgent & High-Priority Emails + Deadline-Driven Emails

Task 3: Validate AI-Generated Results Using the "LLM as a Judge" Technique
To ensure accuracy and reliability, apply the "LLM as a Judge" technique to evaluate:
✅ Relevance: How well does thesummaryaddress the input query or task?
✅ Clarity: How clear and understandable is thesummary?
✅ Actionability: Does thesummaryprovide clear next steps or actionable information?
✅ Strengths: Highlight the key strengths of the summary.
✅ Improvements: Suggest 1-2 areas for improvement.
✅ Overall Justification: Provide a 2-3 line summary evaluation, including key observations.

In [6]:
# Parse JSON content from files
import json
# Use pathlib for convenient path operations
from pathlib import Path

# Always load API_KEY from the local `config.json` file; ignore environment variables
cfg_path = Path('config.json')
API_KEY = None
# Check that the config file exists before attempting to read it
if cfg_path.exists():
    try:
        # Open and parse the config file as JSON
        with open(cfg_path, 'r', encoding='utf-8') as f:
            cfg = json.load(f)
        # Look for common key names and take the first non-empty value
        API_KEY = cfg.get('API_KEY') or cfg.get('api_key') or cfg.get('openai_api_key')
    except Exception:
        # If reading or parsing fails, leave API_KEY as None
        API_KEY = None
else:
    # Inform the user if the file is missing
    print('config.json not found in the current directory.')

# Provide a short confirmation but do not print the secret itself
if API_KEY:
    print(f'API_KEY loaded from config.json (length {len(API_KEY)})')
else:
    print('API_KEY not found in config.json. Add it to the file.')

API_KEY loaded from config.json (length 67)


In [7]:
# Print a masked form of the API key so the full secret is not exposed
# If API_KEY is set, show first 4 and last 4 characters separated by '...'; otherwise print None
api_key_val = globals().get('API_KEY')
if api_key_val:
    print(api_key_val[:8] + '...' + api_key_val[-8:])
else:
    print(api_key_val)

gl-U2Fsd...oxDCQTE2


In [1]:
!pip install -q openai==1.61.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 23.7 MB/s eta 0:00:00


In [2]:
import openai
print(openai.__version__)

1.61.1


In [9]:
# @title Loading the `config.json` file
import json, os

# Load the JSON file and extract values
file_name = 'config.json'
with open(file_name, 'r') as file:
    config = json.load(file)
    os.environ['OPENAI_API_KEY']  = config.get("API_KEY") # Loading the API Key
    os.environ["OPENAI_BASE_URL"] = config.get("OPENAI_API_BASE") # Loading the API Base Url

In [10]:
model_name = "gpt-4o-mini"

In [12]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

In [13]:
# @title LLM function
# @markdown Once the API details are filled, the notebook will automatically load the configuration, and learners can generate model outputs using the llm() function.


def llm(system_prompt, user_prompt):
  try:
      # Craft the messages to pass to chat.completions.create
      prompt = [
          {'role':'system', 'content': system_prompt},
          {'role': 'user', 'content': user_prompt}
      ]

      response = client.chat.completions.create(
          model=model_name,
          messages=prompt,
          temperature=0
      )

      return response.choices[0].message.content.strip()

  except Exception as e:
      prediction = f'Sorry, I encountered the following error: \n {e}'
      print(prediction)

In [ ]:
# @title Step 1: Load the Dataset
# Data Loading

import pandas as pd
# Use the raw GitHub URL for the CSV file
df = pd.read_csv("https://raw.githubusercontent.com/linufx2208-sketch/JHU_Learnings/main/Week%209%20Project/Alex_emails_march_04.csv", index_col="email_id", encoding='latin-1')      #Add the data file location
df

In [18]:
# @title Step 2: Apply Yesterbox Filtering
# @markdown The Yesterbox approach involves processing emails from the previous day first before tackling today's emails.

# @markdown For this dataset, consider today's date as 4th March 2025.

# @markdown We filter the dataset to only include emails received on 3rd March 2025 (yesterday)
# (Yesterbox Approach)(Today: 4 march)



from datetime import datetime, timedelta

yesterday_date = pd.to_datetime("3/3/2025").strftime('%m/%d/%Y')

df['date_received'] = pd.to_datetime(df['date_received']).dt.strftime('%m/%d/%Y')

yesterday_emails = df[df['date_received'] == yesterday_date].reset_index(drop=True)
print(f"Filtered Emails Count: {len(yesterday_emails)}")


Filtered Emails Count: 51
